# Verl PPO Training - Password Game

This notebook implements PPO training for the **multi-turn password game** where:
- Rules accumulate (each turn adds a new constraint)
- Model must satisfy ALL previous rules + current rule
- Reward shaped to encourage progress while minimizing password length
- Single H100, no DDP
- Qwen3-0.6B with thinking mode support

## Key Features

1. **Multi-turn Rollouts**: Handles cumulative rule nature
2. **Shaped Rewards**: Progress bonus + length penalty + success bonus
3. **Episode-based**: Full game runs as episodes
4. **Thinking Mode**: Qwen's thinking capabilities for complex reasoning
5. **Adaptive KL**: Dynamic KL coefficient adjustment

## 🛠️ Setup

In [ ]:
# Check GPU
!nvidia-smi -L

In [ ]:
# Install dependencies
!pip install -q torch==2.4.0 torchvision==0.19.0 --index-url https://download.pytorch.org/whl/cu121
!pip install -q flash-attn --no-build-isolation
!pip install -q transformers>=4.45.0 accelerate datasets tokenizers
!pip install -q wandb tensorboard pandas matplotlib tqdm requests

In [ ]:
import sys
import os
sys.path.append('/home/user/notebooks/RL')

# Verify password game is accessible
sys.path.append('/home/user/notebooks/tasks/password-game')

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 🔑 API Keys

In [ ]:
# Import keys
try:
    from keys import WANDB_API_KEY, HF_TOKEN, OPENAI_API_KEY
    if WANDB_API_KEY:
        os.environ['WANDB_API_KEY'] = WANDB_API_KEY
    if HF_TOKEN:
        os.environ['HF_TOKEN'] = HF_TOKEN
    if OPENAI_API_KEY:
        os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    print("✓ Keys imported")
except ImportError:
    print("⚠️ keys.py not found - some features may not work")

In [ ]:
import wandb
from huggingface_hub import login

# Login to services
try:
    wandb.login()
    print("✓ WandB login successful")
except Exception as e:
    print(f"⚠️ WandB: {e}")

try:
    if os.environ.get('HF_TOKEN'):
        login(token=os.environ['HF_TOKEN'])
        print("✓ HuggingFace login successful")
except Exception as e:
    print(f"⚠️ HF: {e}")

## 🧪 Test Password Game Environment

In [ ]:
from game import PasswordGame

# Test the password game
game = PasswordGame()
print("Password Game Test")
print("="*80)
print(f"Current rule: {game.get_current_rule()}")
print(f"\nGame state: {game.game_active}")
print(f"\nDynamic values:")
print(f"  Captcha: {game.captcha}")
print(f"  Country: {game.country}")
print(f"  Wordle: {game.wordle_answer}")
print(f"  Moon: {game.moon_phase}")

# Test reward calculation
test_password = "Hello123!"
feedback = game.get_rule_feedback(test_password)
print(f"\nTest password: {test_password}")
print(f"Rules passing: {feedback['total_passing']} / {len(feedback['rules_checked'])}")
print(f"Reward: {feedback['reward']}")

## ⚙️ Configuration

Adjust these settings for your training run.

In [ ]:
from verl_password_game_ppo import PasswordGamePPOConfig
import time

# Create config
config = PasswordGamePPOConfig(
    # Model
    model_name="Qwen/Qwen2.5-0.6B",
    precision="bfloat16",
    use_flash_attn=True,
    
    # Training
    num_epochs=5,
    num_episodes_per_epoch=100,
    batch_size=4,
    samples_per_state=2,
    learning_rate=5e-7,
    
    # PPO
    ppo_epochs=4,
    clip_range=0.2,
    value_loss_coef=0.5,
    kl_coef=0.1,
    gamma=0.99,
    
    # Thinking mode
    enable_thinking=True,
    thinking_temperature=0.7,
    parse_thinking=True,
    
    # Password game
    max_turns_per_episode=10,
    reward_scale=1.0,
    length_penalty_scale=0.1,
    progress_bonus=2.0,
    final_success_bonus=10.0,
    
    # Logging
    wandb_project="verl-password-game",
    wandb_run_name=f"ppo_qwen_thinking_{int(time.time())}",
    log_interval=5,
    eval_interval=25,
    save_interval=50,
    output_dir=f"./verl_password_game_{int(time.time())}",
    
    seed=42
)

print("Configuration:")
print(f"  Model: {config.model_name}")
print(f"  Thinking mode: {config.enable_thinking}")
print(f"  Episodes/epoch: {config.num_episodes_per_epoch}")
print(f"  Batch size: {config.batch_size}")
print(f"  Learning rate: {config.learning_rate}")
print(f"  Output: {config.output_dir}")

## 🚀 Run Training

**Note**: This will take several hours depending on your configuration.

In [ ]:
from verl_password_game_ppo import train_ppo_password_game

# Start training
train_ppo_password_game(config)

## 📊 Analyze Results

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

# Load summary
summary_path = os.path.join(config.output_dir, "summary.json")
if os.path.exists(summary_path):
    with open(summary_path) as f:
        summary = json.load(f)
    
    print("Training Summary")
    print("="*80)
    print(f"Baseline reward: {summary['baseline']['mean_reward']:.4f}")
    print(f"Final reward: {summary['final']['mean_reward']:.4f}")
    print(f"Improvement: {summary['improvement']:.4f}")
    print(f"\nBaseline rules: {summary['baseline']['mean_rules']:.2f}")
    print(f"Final rules: {summary['final']['mean_rules']:.2f}")
    print(f"\nFinal success rate: {summary['final']['success_rate']:.2%}")
else:
    print("Summary not found. Training may not be complete.")

## 🧪 Test Trained Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from verl_password_game_ppo import PasswordGameEnv, extract_password_from_response, parse_thinking_response

# Load best model
best_model_path = os.path.join(config.output_dir, "best_model")

if os.path.exists(best_model_path):
    print(f"Loading model from {best_model_path}...")
    
    tokenizer = AutoTokenizer.from_pretrained(best_model_path, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    model = AutoModelForCausalLM.from_pretrained(
        best_model_path,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
        attn_implementation="flash_attention_2"
    )
    model.eval()
    model.config.use_cache = True
    
    print("✓ Model loaded")
else:
    print(f"Model not found at {best_model_path}")
    print("Using untrained model for demonstration...")
    tokenizer = AutoTokenizer.from_pretrained(config.model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(
        config.model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True
    )
    model.eval()

In [ ]:
# Run interactive game
def play_game_interactive(model, tokenizer, config, max_turns=10):
    """Play one game and show step-by-step results."""
    env = PasswordGameEnv(tokenizer, config)
    prompt, info = env.reset()
    
    print("="*80)
    print("PASSWORD GAME - INTERACTIVE PLAYTHROUGH")
    print("="*80)
    
    gen_config = {
        "max_new_tokens": config.max_new_tokens,
        "do_sample": True,
        "temperature": 0.7,
        "top_p": 0.9,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": tokenizer.eos_token_id,
    }
    
    total_reward = 0
    
    for turn in range(max_turns):
        print(f"\n{'='*80}")
        print(f"TURN {turn + 1}")
        print(f"{'='*80}")
        
        # Show current rules
        rules = env.game.get_all_rules_up_to_current()
        print(f"\nRules to satisfy ({len(rules)}):")
        for i, rule in enumerate(rules, 1):
            print(f"  {i}. {rule}")
        
        # Generate response
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(**inputs, **gen_config)
        
        generated_ids = outputs[:, inputs.input_ids.size(1):]
        
        if config.enable_thinking:
            thinking, response = parse_thinking_response(generated_ids[0], tokenizer)
            if thinking:
                print(f"\n💭 Thinking: {thinking[:200]}..." if len(thinking) > 200 else f"\n💭 Thinking: {thinking}")
        else:
            response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        
        password = extract_password_from_response(response)
        print(f"\n🔑 Password: {password}")
        print(f"   Length: {len(password)}")
        
        # Step environment
        next_prompt, reward, done, step_info = env.step(response)
        total_reward += reward
        
        print(f"\n📊 Feedback:")
        print(f"   Rules passing: {step_info['rules_passing']} / {step_info['current_rule_index'] + 1}")
        print(f"   Reward: {reward:.2f}")
        print(f"   Total reward: {total_reward:.2f}")
        
        # Show which rules pass/fail
        feedback = step_info['feedback']
        for rule_info in feedback['rules_checked']:
            status = "✓" if rule_info['passes'] else "✗"
            print(f"   {status} Rule {rule_info['rule_index'] + 1}")
        
        if done:
            print(f"\n{'='*80}")
            print("GAME OVER")
            print(f"{'='*80}")
            print(f"Final turn: {turn + 1}")
            print(f"Rules satisfied: {step_info['rules_passing']}")
            print(f"Total reward: {total_reward:.2f}")
            print(f"Final password: {password}")
            print(f"Final length: {len(password)}")
            break
        
        prompt = next_prompt
    
    return total_reward, step_info['rules_passing']

# Play one game
reward, rules = play_game_interactive(model, tokenizer, config, max_turns=10)

## 🔬 Batch Evaluation

In [ ]:
from verl_password_game_ppo import evaluate_policy

# Evaluate on multiple episodes
print("Running batch evaluation...")
eval_results = evaluate_policy(
    policy_model=model,
    tokenizer=tokenizer,
    config=config,
    num_episodes=20,
    desc="Batch Eval"
)

print("\nBatch Evaluation Results:")
print("="*80)
print(f"Mean reward: {eval_results['mean_reward']:.4f} ± {eval_results['std_reward']:.4f}")
print(f"Mean rules satisfied: {eval_results['mean_rules']:.2f}")
print(f"Max rules satisfied: {eval_results['max_rules']}")
print(f"Success rate (≥5 rules): {eval_results['success_rate']:.2%}")
print(f"Episodes evaluated: {eval_results['num_episodes']}")

## 📈 Visualize Training Progress (from WandB)

If you have WandB data, you can analyze it here.

In [ ]:
# This cell requires wandb API
# Uncomment and modify if you want to pull data from WandB

# import wandb
# api = wandb.Api()
# run = api.run(f"{config.wandb_project}/{config.wandb_run_name}")
# history = run.history()
# 
# # Plot training metrics
# fig, axes = plt.subplots(2, 2, figsize=(15, 10))
# 
# axes[0, 0].plot(history['train/reward'])
# axes[0, 0].set_title('Training Reward')
# axes[0, 0].set_xlabel('Step')
# axes[0, 0].set_ylabel('Reward')
# 
# axes[0, 1].plot(history['train/rules_satisfied'])
# axes[0, 1].set_title('Rules Satisfied')
# axes[0, 1].set_xlabel('Step')
# axes[0, 1].set_ylabel('Rules')
# 
# axes[1, 0].plot(history['train/kl_div'])
# axes[1, 0].set_title('KL Divergence')
# axes[1, 0].set_xlabel('Step')
# axes[1, 0].set_ylabel('KL')
# 
# axes[1, 1].plot(history['train/loss'])
# axes[1, 1].set_title('Total Loss')
# axes[1, 1].set_xlabel('Step')
# axes[1, 1].set_ylabel('Loss')
# 
# plt.tight_layout()
# plt.show()